In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- Beállítások ---
files = [
    "CERV-F3-2024_89db_text_EU_counts.xlsx",
    "CERV-P-2023_93db_text_EU_counts.xlsx",
    "F_2023_FK02_259db_text_EU_counts.xlsx",
]

EU_COLS = [
    "meltosag",
    "szabadsagok",
    "egyenloseg",
    "szolidaritas",
    "polgarok_jogai",
    "igazsagszolgaltatas",
]

def add_global_tercile_codes(df, cols):
    """
    Globális tercilis-kódok:
      - összes érték a 6 oszlopban egy közös listába kerül
      - erre számoljuk a 33% és 66% kvantiliseket
      - minden cellát ezek szerint kódolunk 1 / 2 / 3-ra
    """
    # összes releváns érték egy tömbben
    values = df[cols].to_numpy().ravel()

    # 33% és 66% kvantilis
    q1, q2 = np.quantile(values, [1/3, 2/3])

    print("Globális tercilis-határok:")
    print(f"  1. tercilis vége (q1): {q1:.2f}")
    print(f"  2. tercilis vége (q2): {q2:.2f}")

    for col in cols:
        df[col] = np.select(
            [df[col] <= q1, df[col] <= q2],
            [1, 2],
            default=3
        ).astype(int)

    return df

for fname in files:
    path = Path(fname)
    print(f"Feldolgozás: {path.name}")

    # 1) beolvasás
    df = pd.read_excel(path)

    # 2) globális tercilis kódok
    df_codes = add_global_tercile_codes(df.copy(), EU_COLS)

    # csak az Azonosító + kódolt oszlopok
    keep_cols = ["Azonosító"] + EU_COLS
    df_codes = df_codes[keep_cols]

    # 3) új fájlnév
    new_name = path.with_name(path.stem + "_global_terciles.xlsx")

    # 4) mentés
    df_codes.to_excel(new_name, index=False)
    print(f"  → elmentve: {new_name.name}")

print("Kész: 3 új Excel globális tercilis kódokkal.")


Feldolgozás: CERV-F3-2024_89db_text_EU_counts.xlsx
Globális tercilis-határok:
  1. tercilis vége (q1): 6.00
  2. tercilis vége (q2): 16.00
  → elmentve: CERV-F3-2024_89db_text_EU_counts_global_terciles.xlsx
Feldolgozás: CERV-P-2023_93db_text_EU_counts.xlsx
Globális tercilis-határok:
  1. tercilis vége (q1): 5.00
  2. tercilis vége (q2): 17.00
  → elmentve: CERV-P-2023_93db_text_EU_counts_global_terciles.xlsx
Feldolgozás: F_2023_FK02_259db_text_EU_counts.xlsx
Globális tercilis-határok:
  1. tercilis vége (q1): 5.00
  2. tercilis vége (q2): 16.00
  → elmentve: F_2023_FK02_259db_text_EU_counts_global_terciles.xlsx
Kész: 3 új Excel globális tercilis kódokkal.


In [3]:
pd.DataFrame({
    "percentilis": ["33%", "66%"],
    "ertek": [q1, q2]
}).to_excel("global_tercile_thresholds.xlsx", index=False)


NameError: name 'q1' is not defined